# **Model_1 Prediction**

In this notebook, I'll perform the prediction with Model_1 and compare it with the base model **This Notebook have prediction with Model-1.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import required modules

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn import metrics
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import AdaBoostRegressor,VotingRegressor,RandomForestRegressor,GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

# **In Model-1 :**
 we will **add 10 columns of previous fog index** , because from data analysis we got to know that the future fog index value is **dependent on its previous values**.

# **6 hour Prediction**

In [62]:
# for 6 hour

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-1)

# Here we are adding previous data points as input for prediction
df['fog_index_1'] =  df['fog_index'].shift(1)
df['fog_index_2'] =  df['fog_index'].shift(2)
df['fog_index_3'] =  df['fog_index'].shift(3)
df['fog_index_4'] =  df['fog_index'].shift(4)
df['fog_index_5'] =  df['fog_index'].shift(5)
df['fog_index_6'] =  df['fog_index'].shift(6)
df['fog_index_7'] =  df['fog_index'].shift(7)
df['fog_index_8'] =  df['fog_index'].shift(8)
df['fog_index_9'] =  df['fog_index'].shift(9)
df['fog_index_10'] =  df['fog_index'].shift(10)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-1)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(1)
val['fog_index_2'] =  val['fog_index'].shift(2)
val['fog_index_3'] =  val['fog_index'].shift(3)
val['fog_index_4'] =  val['fog_index'].shift(4)
val['fog_index_5'] =  val['fog_index'].shift(5)
val['fog_index_6'] =  val['fog_index'].shift(6)
val['fog_index_7'] =  val['fog_index'].shift(7)
val['fog_index_8'] =  val['fog_index'].shift(8)
val['fog_index_9'] =  val['fog_index'].shift(9)
val['fog_index_10'] =  val['fog_index'].shift(10)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]

stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)



In [63]:
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))

RandomforestRegressor
0.0009716725942509898
0.011555998306102603
LinearRegressor
0.009245414377130789
0.015140017943361627


In [64]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

In [65]:
gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

GradientBoosting
0.005128324285169552
0.012053771238426972


In [66]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



Adaboost
0.0002986638715289051
0.013008680477580185


In [67]:
# voting ensemble
from sklearn.ensemble import VotingRegressor

estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]

vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.002413996478706164
0.011971826967483166


In [68]:
validate['pred']=  rf.predict(X_val)

In [69]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

# **For 1 day**

In [82]:
# for 1 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-4)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(4)
df['fog_index_2'] =  df['fog_index'].shift(8)
df['fog_index_3'] =  df['fog_index'].shift(12)
df['fog_index_4'] =  df['fog_index'].shift(16)
df['fog_index_5'] =  df['fog_index'].shift(20)
df['fog_index_6'] =  df['fog_index'].shift(24)
df['fog_index_7'] =  df['fog_index'].shift(28)
df['fog_index_8'] =  df['fog_index'].shift(32)
df['fog_index_9'] =  df['fog_index'].shift(36)
df['fog_index_10'] =  df['fog_index'].shift(40)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-4)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(4)
val['fog_index_2'] =  val['fog_index'].shift(8)
val['fog_index_3'] =  val['fog_index'].shift(12)
val['fog_index_4'] =  val['fog_index'].shift(16)
val['fog_index_5'] =  val['fog_index'].shift(20)
val['fog_index_6'] =  val['fog_index'].shift(24)
val['fog_index_7'] =  val['fog_index'].shift(28)
val['fog_index_8'] =  val['fog_index'].shift(32)
val['fog_index_9'] =  val['fog_index'].shift(36)
val['fog_index_10'] =  val['fog_index'].shift(40)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]

In [71]:
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

RandomforestRegressor
0.001620920528357696
0.020491773356295116
LinearRegressor
0.011504554465845094
0.020286424797676412


In [83]:
gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

GradientBoosting
0.0085543819859911
0.021346414858884666


In [84]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



Adaboost
0.0005205503706807648
0.022235037643785642


In [74]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.0037304284847044767
0.02035615197821661


In [75]:
validate['pred']=  rf.predict(X_val)

In [76]:
# with previous data 1 day prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

# **For 3 Day**

In [85]:
# for 3 day Prediction

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-12)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(12)
df['fog_index_2'] =  df['fog_index'].shift(24)
df['fog_index_3'] =  df['fog_index'].shift(36)
df['fog_index_4'] =  df['fog_index'].shift(48)
df['fog_index_5'] =  df['fog_index'].shift(60)
df['fog_index_6'] =  df['fog_index'].shift(72)
df['fog_index_7'] =  df['fog_index'].shift(84)
df['fog_index_8'] =  df['fog_index'].shift(96)
df['fog_index_9'] =  df['fog_index'].shift(108)
df['fog_index_10'] =  df['fog_index'].shift(120)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-12)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(12)
val['fog_index_2'] =  val['fog_index'].shift(24)
val['fog_index_3'] =  val['fog_index'].shift(36)
val['fog_index_4'] =  val['fog_index'].shift(48)
val['fog_index_5'] =  val['fog_index'].shift(60)
val['fog_index_6'] =  val['fog_index'].shift(72)
val['fog_index_7'] =  val['fog_index'].shift(84)
val['fog_index_8'] =  val['fog_index'].shift(96)
val['fog_index_9'] =  val['fog_index'].shift(108)
val['fog_index_10'] =  val['fog_index'].shift(120)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]

In [78]:
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

RandomforestRegressor
0.0020494945657996712
0.027527345628011138
LinearRegressor
0.014761756676250118
0.02678505962636464


In [86]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

gb.fit(X_train,y_train)

print('Gradient Boosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

Gradient Boosting
0.011291326250490633
0.02713993427847481


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



Adaboost
0.0005865778232812392
0.029542555900574193


In [80]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.004848949881494863
0.02707079125897572


In [87]:
validate['pred']=  rf.predict(X_val)

In [88]:
# with previous data 3 day prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

# **For 5 days**

In [89]:
# for 5 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-20)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(20)
df['fog_index_2'] =  df['fog_index'].shift(40)
df['fog_index_3'] =  df['fog_index'].shift(60)
df['fog_index_4'] =  df['fog_index'].shift(80)
df['fog_index_5'] =  df['fog_index'].shift(100)
df['fog_index_6'] =  df['fog_index'].shift(120)
df['fog_index_7'] =  df['fog_index'].shift(140)
df['fog_index_8'] =  df['fog_index'].shift(160)
df['fog_index_9'] =  df['fog_index'].shift(180)
df['fog_index_10'] =  df['fog_index'].shift(200)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-20)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(20)
val['fog_index_2'] =  val['fog_index'].shift(40)
val['fog_index_3'] =  val['fog_index'].shift(60)
val['fog_index_4'] =  val['fog_index'].shift(80)
val['fog_index_5'] =  val['fog_index'].shift(100)
val['fog_index_6'] =  val['fog_index'].shift(120)
val['fog_index_7'] =  val['fog_index'].shift(140)
val['fog_index_8'] =  val['fog_index'].shift(160)
val['fog_index_9'] =  val['fog_index'].shift(180)
val['fog_index_10'] =  val['fog_index'].shift(200)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]

In [34]:
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

RandomforestRegressor
0.0022782251401386945
0.02596419075113191
LinearRegressor
0.015950614146154188
47040199.985141


In [90]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

GradientBoosting
0.012527966479206573
0.024558876183929122


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



Adaboost
0.0007438756525560601
0.026462288319940463


In [91]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.0053645302963512935
0.024134965246427168


In [92]:
validate['pred']=  rf.predict(X_val)

In [93]:
# with previous data 5 day prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()